# Bibliotecas

In [1]:
#Importar Bibliotecas
%pip install lazypredict

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")

Note: you may need to restart the kernel to use updated packages.


# Cargar Data

In [2]:
#cargar los datos

df= pd.read_csv('../data/salaries_data.csv')
df_test = pd.read_csv('../data/testeo.csv')

# Explorar Datos

In [3]:
df= df.drop(['salary', 'salary_currency'], axis = 1)


In [4]:
#df = pd.concat([df_inicio,df_test])

In [5]:
df_test.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,US,100,US,L
2,2020,MI,FT,Data Analyst,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,IN,100,IN,L


In [6]:
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
work_year,500.00,2021.41,0.68,2020.00,2021.00,2022.00,2022.00,2022.00
salary_in_usd,500.00,112996.22,69613.75,2859.00,64594.50,103080.00,150000.00,450000.00
remote_ratio,500.00,71.80,40.47,0.00,50.00,100.00,100.00,100.00


In [8]:
df.shape

(500, 9)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           500 non-null    int64 
 1   experience_level    500 non-null    object
 2   employment_type     500 non-null    object
 3   job_title           500 non-null    object
 4   salary_in_usd       500 non-null    int64 
 5   employee_residence  500 non-null    object
 6   remote_ratio        500 non-null    int64 
 7   company_location    500 non-null    object
 8   company_size        500 non-null    object
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [10]:
df_s = df.groupby('job_title')['salary_in_usd'].mean()
df_s.sort_values()

job_title
3D Computer Vision Researcher                5409.00
Product Data Analyst                        20000.00
Computer Vision Engineer                    28226.75
Data Analytics Engineer                     35000.00
Machine Learning Infrastructure Engineer    54217.50
ETL Developer                               54957.00
Big Data Engineer                           57080.57
AI Scientist                                59410.60
Finance Data Analyst                        61896.00
Data Science Consultant                     67874.20
Computer Vision Software Engineer           70000.00
Machine Learning Developer                  78791.00
Head of Machine Learning                    79039.00
BI Data Analyst                             82454.40
Data Science Engineer                       83705.00
Lead Machine Learning Engineer              87932.00
Marketing Data Analyst                      88654.00
Business Data Analyst                       91253.50
Data Analyst                        

In [11]:
#Límites del criterio turkey para identificar outliers

Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1

outliers = df.groupby('job_title')['salary_in_usd'].mean()[(df.groupby('job_title')['salary_in_usd'].mean() < Q1 - 1.5 * IQR) | (df.groupby('job_title')['salary_in_usd'].mean() > Q3 + 1.5 * IQR)]
outliers

job_title
Data Analytics Lead      405000.00
Financial Data Analyst   450000.00
Name: salary_in_usd, dtype: float64

In [12]:
#regla de las 1.5 veces el rango intercuartílico

Q11 = df['salary_in_usd'].quantile(0.25)
Q33 = df['salary_in_usd'].quantile(0.75)
IQRR = Q33 - Q11
upper_limit = Q33 + 1.5 * IQRR
lower_limit = Q33 - 1.5 * IQRR
print("Q11:", Q1)
print("Q33:", Q3)
print("IQRR:", IQRR)
print("Upper Limit for Outliers:", upper_limit)
print("Lower Limit for Outliers:", lower_limit)

Q11: 64594.5
Q33: 150000.0
IQRR: 85405.5
Upper Limit for Outliers: 278108.25
Lower Limit for Outliers: 21891.75


# Division Data para train y testeo

In [13]:
#train = df.iloc[:500, :]
#test = df.iloc[500:, :]
#test = test.drop(['salary_in_usd'], axis = 1)
X = df[['salary_in_usd']]
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

# Ordenar y jerarquizar datos

#### Ordenar columna Experience level segun salarios


In [15]:
df.groupby('experience_level')['salary_in_usd'].mean()

experience_level
EN    59989.57
EX   196078.85
MI    90384.61
SE   136341.79
Name: salary_in_usd, dtype: float64

In [16]:
#con esta columna vamos a crear una función que haga lo siguiente: 
#si tiene la palabra "ml" o "machine learning", que sea "ml"
#si tiene la palabra "anayst" que sea "analyst"
#si tiene la palabra engineer que sea "engineer"
#si tiene la palabra scientist que sea "scientist"
#y si no tienen ninguno de estos, que sea "other"
def job_title(x):
    if 'ml' in x.lower() or 'machine learning' in x.lower():
        return 'ml'
    elif 'analyst' in x.lower():
        return 'analyst'
    elif 'engineer' in x.lower():
        return 'engineer'
    elif 'scientist' in x.lower():
        return 'scientist'
    else:
        return 'other'
#aplicamos la función en la columna job_title con un lambda
df['job_title'] = df['job_title'].apply(lambda x: job_title(x))
df_test['job_title'] = df_test['job_title'].apply(lambda x: job_title(x))


In [17]:
'''# Definir una función que asigna valores numéricos a diferentes rangos de valores
def assign_value(x):
    if x <= 90000:
        return 0
    elif x < 196000 and x > 90000:
        return 1
    elif x < 300000 and x > 196000:
        return 2
    elif x > 300000:
        return 3


# Aplicar la función a la columna "salary_in_usd" utilizando el método "apply"
df['experience_level'] = df['salary_in_usd'].apply(assign_value)
df_test['experience_level'] = df['salary_in_usd'].apply(assign_value)'''

'# Definir una función que asigna valores numéricos a diferentes rangos de valores\ndef assign_value(x):\n    if x <= 90000:\n        return 0\n    elif x < 196000 and x > 90000:\n        return 1\n    elif x < 300000 and x > 196000:\n        return 2\n    elif x > 300000:\n        return 3\n\n\n# Aplicar la función a la columna "salary_in_usd" utilizando el método "apply"\ndf[\'experience_level\'] = df[\'salary_in_usd\'].apply(assign_value)\ndf_test[\'experience_level\'] = df[\'salary_in_usd\'].apply(assign_value)'

#### Ordenar employee_residence segun salarios

In [18]:
'''mean_salary = train.groupby('employee_residence')['salary_in_usd'].mean()

# Ordenar los valores medios de forma descendente
mean_salary_sorted = mean_salary.sort_values(ascending=False)
'''

"mean_salary = train.groupby('employee_residence')['salary_in_usd'].mean()\n\n# Ordenar los valores medios de forma descendente\nmean_salary_sorted = mean_salary.sort_values(ascending=False)\n"

In [19]:
'''# Definir los límites de los rangos
min_salary = train['salary_in_usd'].min()
max_salary = train['salary_in_usd'].max()
amplitude = (max_salary - min_salary) / 50
range_limits = [min_salary + i * amplitude for i in range(51)]

# Crear las categorías numéricas
train['employee_residence'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
test['employee_residence'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
'''

"# Definir los límites de los rangos\nmin_salary = train['salary_in_usd'].min()\nmax_salary = train['salary_in_usd'].max()\namplitude = (max_salary - min_salary) / 50\nrange_limits = [min_salary + i * amplitude for i in range(51)]\n\n# Crear las categorías numéricas\ntrain['employee_residence'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\ntest['employee_residence'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\n"

#### ordenar company_location según salarios

In [20]:
'''train['company_location'].unique()'''

"train['company_location'].unique()"

In [21]:
'''mean_salary = train.groupby('company_location')['salary_in_usd'].mean()

# Ordenar los valores medios de forma descendente
mean_salary_sorted = mean_salary.sort_values(ascending=False)'''

"mean_salary = train.groupby('company_location')['salary_in_usd'].mean()\n\n# Ordenar los valores medios de forma descendente\nmean_salary_sorted = mean_salary.sort_values(ascending=False)"

In [22]:
'''# Definir los límites de los rangos
min_salary = train['salary_in_usd'].min()
max_salary = train['salary_in_usd'].max()
amplitude = (max_salary - min_salary) / 44
range_limits = [min_salary + i * amplitude for i in range(45)]

# Crear las categorías numéricas
train['company_location'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
test['company_location'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
'''

"# Definir los límites de los rangos\nmin_salary = train['salary_in_usd'].min()\nmax_salary = train['salary_in_usd'].max()\namplitude = (max_salary - min_salary) / 44\nrange_limits = [min_salary + i * amplitude for i in range(45)]\n\n# Crear las categorías numéricas\ntrain['company_location'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\ntest['company_location'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\n"

#### ordenar por job title y salarios medios

In [23]:
'''mean_salary = train.groupby('job_title')['salary_in_usd'].mean()

# Ordenar los valores medios de forma descendente
mean_salary_sorted = mean_salary.sort_values(ascending=False)'''

"mean_salary = train.groupby('job_title')['salary_in_usd'].mean()\n\n# Ordenar los valores medios de forma descendente\nmean_salary_sorted = mean_salary.sort_values(ascending=False)"

In [24]:
'''# Definir los límites de los rangos
min_salary = train['salary_in_usd'].min()
max_salary = train['salary_in_usd'].max()
amplitude = (max_salary - min_salary) / 48
range_limits = [min_salary + i * amplitude for i in range(49)]

# Crear las categorías numéricas
train['job_title'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
test['job_title'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)
'''

"# Definir los límites de los rangos\nmin_salary = train['salary_in_usd'].min()\nmax_salary = train['salary_in_usd'].max()\namplitude = (max_salary - min_salary) / 48\nrange_limits = [min_salary + i * amplitude for i in range(49)]\n\n# Crear las categorías numéricas\ntrain['job_title'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\ntest['job_title'] = pd.cut(train['salary_in_usd'], bins=range_limits, labels=False)\n"

In [25]:
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,engineer,140250,US,100,US,M
1,2022,SE,FT,engineer,135000,US,100,US,M
2,2021,MI,FT,analyst,100000,US,100,US,M
3,2021,MI,CT,ml,270000,US,100,US,L
4,2021,MI,FT,engineer,26005,RO,0,US,L


In [26]:
df = pd.get_dummies(df, columns = ['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size'])
df_test = pd.get_dummies(df_test, columns = ['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size'])


In [27]:
'''# Crear una matriz de correlación con todas las variables
corr_matrix = df.corr()

# Ordenar la correlación de mayor a menor con respecto a salary_usd
corr = corr_matrix['salary_in_usd'].sort_values(ascending=False)

# Crear un heatmap con las variables más correlacionadas con salary_usd
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix.loc[corr.index, corr.index], annot=True, cmap='coolwarm')
plt.show()'''

"# Crear una matriz de correlación con todas las variables\ncorr_matrix = df.corr()\n\n# Ordenar la correlación de mayor a menor con respecto a salary_usd\ncorr = corr_matrix['salary_in_usd'].sort_values(ascending=False)\n\n# Crear un heatmap con las variables más correlacionadas con salary_usd\nplt.figure(figsize=(12,10))\nsns.heatmap(corr_matrix.loc[corr.index, corr.index], annot=True, cmap='coolwarm')\nplt.show()"

# Modelo Lazy

In [40]:


# Seleccionar las variables predictoras y la variable objetivo
X = df.drop(['salary_in_usd'], axis = 1)
y = df['salary_in_usd']


# Dividir el dataset en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un modelo de regresión lineal
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# Entrenar el modelo con el conjunto de entrenamiento
models, predictions = reg.fit(X_train, X_test, y_train, y_test)


# Evaluar el modelo usando R2 score
#r2 = r2_score(y_test, y_pred)
#print(f"R2 score: {r2}")
print(models)

100%|██████████| 42/42 [00:05<00:00,  7.44it/s]

                                                            Adjusted R-Squared  \
Model                                                                            
Lars                          698168442509481163906183114995979575195860992.00   
RANSACRegressor                               55597888842277934003424067584.00   
TransformedTargetRegressor                     3175467926990929956649828352.00   
LinearRegression                               3175467926990929956649828352.00   
MLPRegressor                                                             37.64   
LinearSVR                                                                37.56   
KernelRidge                                                              35.31   
GaussianProcessRegressor                                                 14.75   
SVR                                                                      10.15   
QuantileRegressor                                                        10.14   
DummyRegressor  

In [42]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df.drop(['salary_in_usd'], axis = 1)
y = df['salary_in_usd']


mlp = MLPRegressor(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
mlp.fit(X, y)



MLPRegressor(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

In [43]:
X = df.drop(['salary_in_usd'], axis = 1)
y = df['salary_in_usd']

model3 = MLPRegressor()
model3.fit(X, y)

MLPRegressor()

In [44]:
from sklearn.ensemble import GradientBoostingRegressor

# Definir los datos de entrenamiento y prueba
X = df.drop(['salary_in_usd'], axis = 1)
y = df['salary_in_usd']




# Crear el modelo de Gradient Boosting Regressor
model3 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)

# Entrenar el modelo con los datos de entrenamiento
model3.fit(X, y)

# Realizar predicciones en los datos de prueba
#y_pred = model.predict(X_test)
#y_pred

GradientBoostingRegressor()

In [45]:
m3 = model3.predict(df_test)

In [46]:
m3

array([ 1.31673267e+05,  1.38903919e+05,  1.38903919e+05,  7.20021098e+04,
        3.41126344e+04,  8.76857534e+04,  1.05577812e+05,  5.62791158e+04,
        5.12910926e+04,  1.74173998e+05,  4.20037771e+04,  1.96941106e+05,
        6.13069506e+04,  3.51371609e+04,  9.81655664e+04,  5.02029232e+04,
        1.02782066e+05,  1.48990233e+05,  5.13921130e+04,  5.29682278e+04,
        5.52926635e+04,  5.65505289e+04,  7.08975430e+04,  1.41906185e+05,
       -2.01782857e-01,  3.52177786e+04,  9.94680476e+04,  1.71815598e+05,
        6.48718533e+04,  1.33724784e+05,  1.30876076e+05,  1.30559742e+05,
        1.22654075e+05,  5.93000701e+04,  4.70801997e+04,  6.70760140e+04,
        1.71551745e+05,  1.91940869e+04,  6.10963178e+04,  1.67548268e+05,
        2.32877036e+04,  1.84217129e+04,  1.42923930e+05,  1.82014889e+05,
        9.77461142e+04,  2.28588063e+05,  1.30812843e+05,  5.53257946e+04,
        1.30559742e+05,  3.27682820e+04,  1.46727100e+05,  6.03738765e+04,
        1.45921300e+05,  

In [49]:
muestra3['salary_in_usd'] = m3

In [48]:
muestra3 = pd.read_csv('../data/muestra.csv')

In [50]:
muestra3.to_csv('competition4.csv', index = False)

In [32]:
df.columns.difference(df_test.columns)

Index(['company_location_AE', 'company_location_AS', 'company_location_BE',
       'company_location_CH', 'company_location_CL', 'company_location_DZ',
       'company_location_EE', 'company_location_HN', 'company_location_HU',
       'company_location_IE', 'company_location_IR', 'company_location_KE',
       'company_location_LU', 'company_location_MD', 'company_location_MT',
       'company_location_NG', 'company_location_NL', 'company_location_NZ',
       'company_location_PK', 'company_location_PT', 'company_location_RO',
       'company_location_SG', 'company_location_SI', 'company_location_TR',
       'company_location_UA', 'company_location_VN', 'employee_residence_AE',
       'employee_residence_AT', 'employee_residence_BE',
       'employee_residence_BG', 'employee_residence_BO',
       'employee_residence_CH', 'employee_residence_CL',
       'employee_residence_CZ', 'employee_residence_DZ',
       'employee_residence_EE', 'employee_residence_HK',
       'employee_residence_HN

In [33]:
#vamos a crear las columnas que faltan en dftest respecto a df
def create_missing_columns(df, dftest):
    missing_cols = set(df.columns) - set(dftest.columns)
    for c in missing_cols:
        dftest[c] = 0
    dftest = dftest[df.columns]
    return dftest

df_test = create_missing_columns(df, df_test)
#elimino la columna salary_in_usd
df_test.drop(['salary_in_usd'], axis=1, inplace=True)

In [35]:
muestra2 = pd.read_csv('../data/muestra.csv')

In [36]:
m2 = model2.predict(df_test)

In [37]:
muestra2['salary_in_usd'] = m2

In [38]:
muestra2.to_csv('competition3.csv', index = False)

In [ ]:
muestra = pd.read_csv('../data/muestra.csv')

In [ ]:
m = model2.predict(df_test)

In [ ]:
muestra['salary_in_usd'] = m2

In [ ]:
muestra.to_csv('competition2.csv', index = False)

In [52]:
muestra3.head()

,id,salary_in_usd
0,0,131673.27
1,1,138903.92
2,2,138903.92
3,3,72002.11
4,4,34112.63
